In [1]:
print('e')

e


In [1]:
import re
import PyPDF2
import io
import google.generativeai as genaiI
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from concurrent.futures import ThreadPoolExecutor
import os
import tempfile
import json
from langchain_community.document_loaders import PyPDFLoader
# from langchain_google_genai import GoogleGenerativeAIEmbeddings


c:\Users\UTKARSH\.conda\envs\aiagent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pdf_path = "C:\AI-Recruitment-agent\datascienceresume32 - Google Docs.pdf"

In [3]:
loder = PyPDFLoader(pdf_path)


In [4]:
documents = loder.load()

# print(documents[0].page_content)


In [5]:
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEYY = os.getenv("GEMINI_API")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEYY)


In [46]:
result = llm.invoke('hii')
print(result.content)

Hi there! How can I help you today?



In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEYY)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Maximum size of each chunk
    chunk_overlap=200,    # Overlap between chunks to maintain context
    length_function=len  # Function to measure chunk length
    
)
chunks = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in chunks]  # Extract text from Document objects
vectorstore = FAISS.from_texts(texts, embeddings)



In [8]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {input} 
""")
     



In [9]:
retriever = vectorstore.as_retriever()


In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [11]:
from langchain.chains import create_retrieval_chain
ra_chain = create_retrieval_chain(retriever, document_chain)


In [12]:
response = ra_chain.invoke({"input": 'What is my fathers name?'})

In [53]:
print(response['answer'])

I'm sorry, but this document does not contain information about the father's name.



In [15]:
print(response['context'][0].page_content)

●  Data Science Certification  -  Coursera  November 2023 
 ●  Earned Badge of Python and SQL  –  HackerRank  January 2024 
 ●  Five Star Python Coder -  HackerRank  May 2024 
 ADDITIONAL INFORMA TION 
 ●  Location :  Delhi 
 ●  Availability :  Immediate Joiner


In [16]:
skills_to_analyze = [
    "Python",
    "Java",
    "React",
    "PostgreSQL",
    "Cloud Computing (AWS/Azure/GCP)",
    "Leadership",
    "Project Management",
    "Machine Learning" # Example of a skill that might not be explicitly mentioned
]


In [21]:
# skill_scores = []
# for skill in skills_to_analyze:
#     query = f"On a scale of 0-10, how clearly does the candidate mention proficiency in {skill}? Provide a numeric rating first, followed by resoning"
#     response = retrieval_chain.invoke({"input": query})
#     match = re.search(r'(\d+)', response['answer'])
#     score = int(match.group(1) if match else 0)
#     reasoning_parts = response['answer'].split('\n', 1)
#     if len(reasoning_parts) > 1:
            
#             reasoning = reasoning_parts[1].strip()
#     else:
#             reasoning = response['answer'].strip()
#     # reasoning = response['answer'].split('\n', 1)[1].strip() if '.' in response['answer'] and len(response['answer'].split('.')) > 1 else ""
#     final_score = min(score, 10)
#     skill_scores.append(final_score)
#     print(f"  Score for {skill}: {final_score}/10")
#     print(f"  Reasoning: {reasoning}")
# # Overall score calculation
    
# # overall_score = sum(skill_scores)
# # n = len(skill_scores)
# # print(f"Overall Score: {overall_score}/{n * 10}")
# # find overll score
# print(skill_scores)





In [58]:
# final code
skill_scores = []
for skill in skills_to_analyze:
    query = f"On a scale of 0-10, how clearly does the candidate mention proficiency in {skill}? Provide a numeric rating first, followed by resoning"
    response = ra_chain.invoke({"input": query})
    match = re.search(r'(\d+)', response['answer'])
    score = int(match.group(1) if match else 0)
    reasoning_parts = response['answer']
    reasining_lines = [line.strip() for line in reasoning_parts.split('\n') if line.strip()]
    
    raw_reasoning = ' '.join(reasining_lines)
   
    final_resoning = re.sub(r'^\d+\s*[-.:]?\s*', '', raw_reasoning)
    # if len(reasoning_parts) > 1:
            
    #         reasoning = reasoning_parts[1].strip()
    # else:
    #         reasoning = response['answer'].strip()
    # reasoning = response['answer'].split('\n', 1)[1].strip() if '.' in response['answer'] and len(response['answer'].split('.')) > 1 else ""
    final_score = min(score, 10)
    skill_scores.append(final_score)
    print(f"  Score for {skill}: {final_score}/10")
    print(f"  Reasoning: {final_resoning}")
# Overall score calculation
    
overall_score = sum(skill_scores)
n = len(skill_scores)
print(f"Overall Score: {overall_score}/{n * 10}")



  Score for Python: 10/10
  Reasoning: The candidate explicitly lists Python under "Programming Languages," earned a "Five Star Python Coder" badge, and has multiple projects and experiences demonstrating Python skills.
  Score for Java: 0/10
  Reasoning: The document does not mention Java at all.
  Score for React: 0/10
  Reasoning: React is not mentioned in the provided context.
  Score for PostgreSQL: 0/10
  Reasoning: The candidate does not mention PostgreSQL.
  Score for Cloud Computing (AWS/Azure/GCP): 0/10
  Reasoning: The candidate does not mention any Cloud Computing skills (AWS/Azure/GCP) in the provided context.
  Score for Leadership: 0/10
  Reasoning: The candidate does not mention leadership skills or experience in the provided context.
  Score for Project Management: 0/10
  Reasoning: The candidate does not mention proficiency in Project Management.
  Score for Machine Learning: 10/10
  Reasoning: /10. The candidate mentions Machine Learning under "Technical Skills", and

In [13]:
weaknesses  = []
improvement_suggestions = {}

analysis_result = {
    "missing_skill": ["Mlops", "Devops", 'GenerativeAI']
}

for skill in analysis_result.get('missing_skill', []):
    prompt = f"""
    Analyze why the resume is weak in demonstrating proficiency in "{skill}".

    For your analysis, consider:
    1. What's missing from the resume regarding this skill?
    2. How could it be improved with specific examples?
    3. What specific action items would make this skill stand out?

    Provide your response in this JSON format:
    {{
        "weakness": "A concise description of what's missing or problematic (1-2 sentences)",
        "improvement_suggestions": [
            "Specific suggestion 1",
            "Specific suggestion 2",
            "Specific suggestion 3"
        ],
        "example_addition": "A specific bullet point that could be added to showcase this skill"
    }}

    Return only valid JSON, no other text.
    """
    
    response = ra_chain.invoke({"input": prompt})
    
    # ✅ Extract string answer
    raw_json = response['answer']

    # ✅ Remove backticks and 'json' label
    cleaned_json = re.sub(r'^```json|```$', '', raw_json.strip(), flags=re.MULTILINE).strip()

    try:
        # ✅ Parse string into dictionary
        weakness_data = json.loads(cleaned_json)

        # ✅ Store in desired format
        weakness_detail = {
            "skill": skill,
            "detail": weakness_data.get("weakness", "No specific details provided."),
            "suggestions": weakness_data.get("improvement_suggestions", []),
            "example": weakness_data.get("example_addition", "")
        }

        weaknesses.append(weakness_detail)
        improvement_suggestions[skill] = {
                    "suggestions": weakness_data.get("improvement_suggestions", []),
                    "example": weakness_data.get("example_addition", "")}

    except json.JSONDecodeError as e:
        print(f"JSON parsing failed for skill {skill}: {e}")
        weaknesses.append({
            "skill": skill,
            "detail": raw_json[:200],  # fallback: first 200 characters
            
            "example": ""
        })

resume_weakness = weaknesses

In [14]:
for item in resume_weakness:
    print(f"\n🔧 Skill: {item['skill']}")
    print(f"❌ Weakness: {item['detail']}")
    print(f"📌 Example: {item['example']}")
    print("✅ Suggestions:")
    for s in item['suggestions']:
        print(f"  - {s}")


🔧 Skill: Mlops
❌ Weakness: The resume lacks explicit mention of MLOps practices. While some projects hint at components of MLOps (model deployment, version control), there's no direct evidence of a systematic approach to model building, deployment, monitoring, and maintenance.
📌 Example: Implemented CI/CD pipelines using Jenkins to automate the deployment of the Gemstone Price Predictor model to a cloud-based platform, reducing deployment time by 50% and enabling continuous model monitoring.
✅ Suggestions:
  - Explicitly list MLOps tools and platforms used (e.g., Docker, Kubernetes, CI/CD pipelines, model monitoring tools).
  - Detail the deployment strategy for any of the models built, highlighting automation, scaling, and monitoring aspects.
  - Quantify the impact of MLOps practices, such as reduced deployment time or improved model performance after deployment.

🔧 Skill: Devops
❌ Weakness: The resume is completely missing any mention of DevOps related skills, tools, or experience.

In [78]:
for skill, suggestion_data in improvement_suggestions.items():
    print(f"\n🔧 Skill: {skill}")
    
    print("✅ Suggestions:")
    for suggestion in suggestion_data.get("suggestions", []):
        print(f"  - {suggestion}")
    
    print(f"📌 Example addition: {suggestion_data.get('example', 'No example provided.')}")



🔧 Skill: Mlops
✅ Suggestions:
  - Mention experience with containerization tools like Docker or Kubernetes.
  - Include experience with CI/CD pipelines for model deployment.
  - Add information about model monitoring and performance tracking tools.
📌 Example addition: Implemented CI/CD pipelines using Jenkins to automate model retraining and deployment, reducing deployment time by 50% and ensuring consistent performance monitoring.

🔧 Skill: Devops
✅ Suggestions:
  - Include any experience with cloud platforms (AWS, Azure, GCP) that might involve infrastructure management or deployment.
  - Mention any experience with containerization technologies like Docker or orchestration tools like Kubernetes.
  - If any projects involved automation of deployment or testing, highlight those aspects.
📌 Example addition: Automated deployment of the Gemstone Price Predictor using Docker and GitHub Actions, resulting in a 20% reduction in deployment time.

🔧 Skill: GenerativeAI
✅ Suggestions:
  - Inc

In [147]:
def extract_skills_jd(jd_text):
    "Extract skills from job description"
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEYY)
    jd_text = """
    Job Title: Data Scientist
    Location: Remote / Bangalore, India
    Job Type: Full-time

    About the Role:
    We are seeking a Data Scientist who is passionate about using data to solve real-world problems and drive strategic decisions. You'll work closely with business, product, and engineering teams to uncover insights, build predictive models, and design experiments that impact millions of users.

    Key Responsibilities:
    - Analyze structured and unstructured data from multiple sources to extract meaningful insights.
    - Build machine learning models for prediction, classification, segmentation, and recommendation.
    - Design A/B tests and evaluate the performance of models and features.
    - Communicate findings clearly to stakeholders using dashboards, reports, and visualizations.
    - Collaborate with data engineers and software developers to deploy scalable data solutions.
    - Monitor model performance and perform periodic model retraining.

    Required Skills & Qualifications:
    - Bachelor's or Master's degree in Computer Science, Statistics, Mathematics, or related field.
    - 2+ years of experience in data science or a similar role.
    - Strong knowledge of Python, SQL, and machine learning libraries like scikit-learn, TensorFlow, or PyTorch.
    - Hands-on experience with data analysis and visualization tools such as Pandas, Matplotlib, Seaborn, or Tableau.
    - Understanding of statistical modeling, hypothesis testing, and experimental design.
    - Experience working with large datasets and cloud platforms (AWS, GCP, or Azure).
    """

    prompt = f"""
                Extract a comprehensive list of technical skills, technologies, and competencies required from this job description. 
                Format the output as a Python list of strings. Only include the list, nothing else.
                
                Job Description:
                {jd_text}
                """

    response = llm.invoke(prompt)
    skill_text = response.content
    cleaned = re.sub(r"```python|```", "",skill_text).strip()

    jd_skills = eval(cleaned)
    return jd_skills



In [148]:
print(extract_skills_jd(jd_text))


['Data Analysis', 'Machine Learning', 'Predictive Modeling', 'Classification', 'Segmentation', 'Recommendation Systems', 'A/B Testing', 'Model Evaluation', 'Data Visualization', 'Dashboard Creation', 'Report Generation', 'Statistical Modeling', 'Hypothesis Testing', 'Experimental Design', 'Model Deployment', 'Model Monitoring', 'Model Retraining', 'Python', 'SQL', 'scikit-learn', 'TensorFlow', 'PyTorch', 'Pandas', 'Matplotlib', 'Seaborn', 'Tableau', 'Cloud Platforms (AWS, GCP, or Azure)', 'Working with Large Datasets', 'Communication', 'Collaboration', 'Data Engineering', 'Software Development', 'Analyzing structured data', 'Analyzing unstructured data', 'Extracting meaningful insights']


In [15]:
resume_text = """
Utkarsh Raj
Email: utkarsh@example.com | Phone: +91-9876543210 | Location: Rohtak, Haryana, India
LinkedIn: www.linkedin.com/in/utkarsh9911 | GitHub: github.com/utkarshraj99

Objective:
Aspiring Data Scientist with hands-on experience in building machine learning models, performing data analysis, and implementing MLOps tools. Seeking to leverage my skills to solve real-world business problems and deliver data-driven solutions.

Education:
Bachelor of Science in Computer Science (2021 – 2025)
Maharshi Dayanand University, Rohtak, Haryana

Skills:
- Programming Languages: Python, SQL
- Libraries & Tools: Pandas, NumPy, Scikit-learn, TensorFlow, Matplotlib, Seaborn, NLTK, SpaCy
- MLOps Tools: DVC, MLflow, Apache Airflow, Docker, Git
- Web Development: Flask, Streamlit, FastAPI
- Cloud Platforms: Google Cloud Storage
- Version Control: Git, GitHub
- Data Engineering: YAML, JSON, CSV handling, Logging & Error Handling

Projects:

1. Stock Price Prediction using LSTM
   - Built and trained an LSTM model using historical Apple stock data via yfinance
   - Performed data preprocessing, scaling, and model evaluation
   - Technologies: TensorFlow, Pandas, Matplotlib

2. Resume Analyzer & Interview Bot
   - Built a Streamlit app that evaluates resumes, gives ATS scores, and suggests improvements
   - Includes features to generate interview questions and analyze job descriptions
   - Integrated RAG-based Q&A with LLMs

3. Customer Lifetime Value Prediction (CLTV)
   - Performed RFM segmentation and applied BetaGeoFitter and GammaGamma models
   - Developed marketing strategy recommendations for each customer cluster

4. Hate Speech Detection (NLP)
   - Created a dataset pipeline with MongoDB, Zip extraction, and preprocessing
   - Built models to classify hate speech in tweets using logistic regression and NLP techniques

Certifications:
- AI For Everyone – Coursera
- Data Science & Machine Learning – Kaggle Learn

Achievements:
- Completed over 10 real-world data science projects
- Contributed to open-source ML templates using DVC and MLflow

Languages:
- English (Intermediate), Hindi (Native)

Declaration:
I hereby declare that the above information is true to the best of my knowledge.
"""


In [16]:
def create_vector_store(text):
    """Create a simpler vector store for skill analysis"""
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEYY)
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Maximum size of each chunk
    chunk_overlap=200,    # Overlap between chunks to maintain context
    length_function=len ) # Function to measure chunk length
    
    chunks = text_splitter.split_text(text)
    vectorstore = FAISS.from_texts(chunks, embeddings)
    return vectorstore

def analyze_skills(ra_chain, skill):
    
    """Analyze a single skill using the RAG chain and return skill, score, and cleaned reasoning."""
        
        # Ask the chain for evaluation of the skill
    query = f"On a scale of 0-10, how clearly does the candidate mention proficiency in {skill}? Provide a numeric rating first, followed by reasoning"
    response = ra_chain.invoke({"input": query})

        # Extract numeric score from the response
    match = re.search(r'(\d+)', response['answer'])
    score = int(match.group(1)) if match else 0
        # final_score = min(score, 10)

        # Clean up the reasoning part
    reasoning_raw = response['answer']
    reasoning_lines = [line.strip() for line in reasoning_raw.split('\n') if line.strip()]
    raw_reasoning = " ".join(reasoning_lines)

        # Remove leading score and optional symbols
    reasoning = re.sub(r'^\d+\s*[-.:]?\s*', '', raw_reasoning)

    return skill, min(score, 10), reasoning

In [70]:
response = qa_chain.invoke({"query": "What skills does the resume show in MLOps?"})

In [35]:
print(response['result'])

Based on the resume, Utkarsh Raj has experience with the following MLOps tools:

*   DVC
*   MLflow
*   Apache Airflow
*   Docker
*   Git


In [17]:
skills = [
    "Python",
    "Java",
    "React",
    "PostgreSQL",
    "Cloud Computing (AWS/Azure/GCP)",
    "Leadership",
    "Project Management",
    "Machine Learning" # Example of a skill that might not be explicitly mentioned
]

In [18]:
skill_scores = {}
skill_reasoning = {}
missing_skills = []
total_score = 0

from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {input} 
""")


vectorstore = create_vector_store(resume_text)
retriever = vectorstore.as_retriever()

     
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)
from langchain.chains import create_retrieval_chain
ra_chain = create_retrieval_chain(retriever, document_chain)

skills_analayze = [
    "Python",
    "Java",
    "React",
    "PostgreSQL",
    "Cloud Computing (AWS/Azure/GCP)",
    "Leadership",
    "Project Management",
    "Machine Learning" # Example of a skill that might not be explicitly mentioned
]


with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(lambda skill: analyze_skills(ra_chain, skill), skills))


for skill, score, reasoning in results:

    skill_scores[skill] = score
    skill_reasoning[skill] = reasoning
    total_score = total_score + score
    if score <=5:
          missing_skills.append(skill)
    
    # print(f"Raw response for {skill}: {response['answer']}")
    
overall_score = int((total_score / (10 *len(skills)))*100)

selected = overall_score >= 50

reasoning = "Candidate evaluated based on explicit resume content using semantic similarity and clear numeric scoring."
strengths = [skill for skill, score in skill_scores.items() if score >= 7]
improvement_areas = missing_skills if not selected else []

print(overall_score)
print(skill_scores)
print(skill_reasoning)
print(selected)
print(reasoning)
print(missing_skills)
print(strengths)
print(improvement_areas)



23
{'Python': 8, 'Java': 0, 'React': 0, 'PostgreSQL': 0, 'Cloud Computing (AWS/Azure/GCP)': 2, 'Leadership': 0, 'Project Management': 0, 'Machine Learning': 9}
{'Python': 'The candidate lists Python under "Programming Languages" in the "Skills" section. While they don\'t explicitly state their proficiency level (e.g., "expert," "proficient"), its inclusion alongside SQL indicates a foundational skill. The listed projects further demonstrate practical application of Python. A higher rating could be warranted with an explicit self-assessment of proficiency.', 'Java': 'Java is not mentioned at all.', 'React': 'React is not mentioned in the document.', 'PostgreSQL': 'Reasoning: PostgreSQL is not mentioned anywhere in the provided text.', 'Cloud Computing (AWS/Azure/GCP)': 'The candidate only mentions Google Cloud Storage. There is no mention of AWS or Azure and their experience with Google Cloud Storage is not detailed.', 'Leadership': 'The candidate does not mention leadership skills in t

In [19]:
weaknesses  = []
improvement_suggestions = {}

analysis_result = {
    "missing_skill": ["Mlops", "Devops", 'GenerativeAI']
}

for skill in analysis_result.get('missing_skill', []):
    prompt = f"""
    Analyze why the resume is weak in demonstrating proficiency in "{skill}".

    For your analysis, consider:
    1. What's missing from the resume regarding this skill?
    2. How could it be improved with specific examples?
    3. What specific action items would make this skill stand out?

    Resume Content:
    {resume_text[:30000]}

    Provide your response in this JSON format:
    {{
        "weakness": "A concise description of what's missing or problematic (1-2 sentences)",
        "improvement_suggestions": [
            "Specific suggestion 1",
            "Specific suggestion 2",
            "Specific suggestion 3"
        ],
        "example_addition": "A specific bullet point that could be added to showcase this skill"
    }}

    Return only valid JSON, no other text.
    """
    
    response = llm.invoke(prompt)
    
    # ✅ Extract string answer
    raw_json = response.content

    # ✅ Remove backticks and 'json' label
    cleaned_json = re.sub(r'^```json|```$', '', raw_json.strip(), flags=re.MULTILINE).strip()

    try:
        # ✅ Parse string into dictionary
        weakness_data = json.loads(cleaned_json)

        # ✅ Store in desired format
        weakness_detail = {
            "skill": skill,
            "detail": weakness_data.get("weakness", "No specific details provided."),
            "suggestions": weakness_data.get("improvement_suggestions", []),
            "example": weakness_data.get("example_addition", "")
        }

        weaknesses.append(weakness_detail)
        improvement_suggestions[skill] = {
                    "suggestions": weakness_data.get("improvement_suggestions", []),
                    "example": weakness_data.get("example_addition", "")}

    except json.JSONDecodeError as e:
        print(f"JSON parsing failed for skill {skill}: {e}")
        weaknesses.append({
            "skill": skill,
            "detail": raw_json[:200],  # fallback: first 200 characters
            
            "example": ""
        })

resume_weakness = weaknesses

In [20]:
for item in resume_weakness:
    print(f"\n🔧 Skill: {item['skill']}")
    print(f"❌ Weakness: {item['detail']}")
    print(f"📌 Example: {item['example']}")
    print("✅ Suggestions:")
    for s in item['suggestions']:
        print(f"  - {s}")


🔧 Skill: Mlops
❌ Weakness: The resume lists MLOps tools but lacks demonstrable application and impact within projects. There's no quantifiable evidence of how MLOps practices improved model deployment, monitoring, or overall project efficiency.
📌 Example: - Implemented a CI/CD pipeline using DVC, MLflow, and Docker for the Stock Price Prediction project, resulting in a 40% reduction in model deployment time and automated model versioning.
✅ Suggestions:
  - Quantify the impact of MLOps tools used in the projects. For example, improved model deployment speed, reduced training time, or enhanced model monitoring capabilities.
  - Describe the specific MLOps workflows implemented (e.g., CI/CD pipeline for model deployment, automated model retraining process, version control for models and data).
  - Focus on the end-to-end MLOps lifecycle, including data validation, model training, deployment, monitoring, and retraining.

🔧 Skill: Devops
❌ Weakness: While the resume mentions DevOps tools 

In [21]:
analysis_result = {'strengths':{'python', 'machine learning'}}
extracted_skills = [
    "Python",
    "Java",
    "React",
    "PostgreSQL",
    "Cloud Computing (AWS/Azure/GCP)",
    "Leadership",
    "Project Management",
    "Machine Learning" 
]
num_questions = 3
difficulty = 'Medium'
question_types = ['technical', 'Behavioural']
context = f"""
Resume Content:
{resume_text[:2000]}...
            
Skills to focus on: {', '.join(extracted_skills)}
            
Strengths: {', '.join(analysis_result.get('strengths', []))}
            
Areas for improvement: {', '.join(missing_skills)}
            """
            
prompt = f"""
Generate {num_questions} personalized {difficulty.lower()} level interview questions for this candidate 
based on their resume and skills. Include only the following question types: {', '.join(question_types)}.
            
For each question:
1. Clearly label the question type
2. Make the question specific to their background and skills
3. For coding questions, include a clear problem statement
            
{context}
            
Format the response as a list of tuples with the question type and the question itself.
Each tuple should be in the format: ("Question Type", "Full Question Text")
"""

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEYY)
response =  llm.invoke(prompt)



In [22]:
print(response.content)
questions_text = response.content

Here are 3 interview questions tailored for Utkarsh Raj, based on his resume and skills:

```python
[
    ("Technical", "In your Stock Price Prediction project, you used LSTM with TensorFlow. Could you describe a situation where a simple, less computationally expensive model like ARIMA might be preferred over LSTM, and explain the trade-offs involved in that decision, considering factors like data characteristics and prediction horizon?"),
    ("Behavioural", "You've listed several projects on your resume, including the 'Resume Analyzer & Interview Bot'. Tell me about a time when you encountered a significant technical challenge while building this project. How did you approach problem-solving, what resources did you utilize, and what did you learn from the experience?"),
    ("Technical", "In your Hate Speech Detection project, you mention creating a dataset pipeline. Let's say you need to productionize this pipeline to handle a continuous stream of tweets. Design a simple architectur

In [23]:
questions_text = response.content
questions = []

# Step 1: Remove any extra explanation text before the list
match = re.search(r"\[\s*\(", questions_text, re.DOTALL)
if match:
    start_index = match.start()
    questions_text = questions_text[start_index:]

# Step 2: Clean up Markdown or Python syntax
questions_text = re.sub(r"```(?:python)?|```", "", questions_text).strip()
print(questions_text)

# Step 3: Safely evaluate the string into a Python list
try:
    questions_list = eval(questions_text)
    questions = []
    for question_type, question in questions_list:
        for requested_type in question_types:
            if requested_type.lower() in question_type.lower():
                questions.append((question_type.strip(), question.strip()))
                break
except Exception as e:
    print(f"Parsing failed: {e}")
    questions = []


[
    ("Technical", "In your Stock Price Prediction project, you used LSTM with TensorFlow. Could you describe a situation where a simple, less computationally expensive model like ARIMA might be preferred over LSTM, and explain the trade-offs involved in that decision, considering factors like data characteristics and prediction horizon?"),
    ("Behavioural", "You've listed several projects on your resume, including the 'Resume Analyzer & Interview Bot'. Tell me about a time when you encountered a significant technical challenge while building this project. How did you approach problem-solving, what resources did you utilize, and what did you learn from the experience?"),
    ("Technical", "In your Hate Speech Detection project, you mention creating a dataset pipeline. Let's say you need to productionize this pipeline to handle a continuous stream of tweets. Design a simple architecture using Apache Airflow to automate the data ingestion, preprocessing, model training, and deployment

In [ ]:
try:
    improvements = {}
    improvement_areas=["Skills Highlighting", "Project Description", "Achievements"]
    target_role="Data Scientist"
    for area in improvement_areas:
        if area == "Skills Highlighting" and resume_weakness:
            skill_improvements = {
                        "description": "Your resume needs to better highlight key skills that are important for the role.",
                        "specific": []
                    }
            


            before_after_examples = {}
            for weaknesses in resume_weakness:
                skill_name = weaknesses.get("skill", "")
                if "suggestions" in weaknesses and weaknesses["suggestions"]:
                        for suggestion in weaknesses["suggestions"]:
                                skill_improvements["specific"].append(f"**{skill_name}**: {suggestion}")

            if 'example' in weaknesses and weaknesses['example']:
                resume_chunks = resume_text.split('\n\n')
                relevant_chunk = ""

                for chunk in resume_chunks:
                    if skill_name.lower() in chunk.lower() or "experience" in chunk.lower():
                        relevant_chunk = chunk
                        break
                if relevant_chunk:
                    before_after_examples = {
                                    "before": relevant_chunk.strip(),
                                    "after": relevant_chunk.strip() + "\n• " + weaknesses["example"]
                                }
        if before_after_examples:
            skill_improvements["before_after"] = before_after_examples
        improvements["Skills Highlighting"] = skill_improvements
            
             
    remaining_areas = [area for area in improvement_areas if area not in improvements]
    if remaining_areas:
        llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEYY)
            
        weaknesses_text = ""
        if resume_weakness:
            weaknesses_text  = "Resume Weaknesses:\n"
        for i, weakness in enumerate(resume_weakness):
            weaknesses_text += f"{i+1}. {weakness['skill']}: {weakness['detail']}\n"
    
            if "suggestions" in weakness:
                for j, sugg in enumerate(weakness["suggestions"]):
                    weaknesses_text += f"   - {sugg}\n"

    context = f"""
                Resume Content:
                {resume_text}
                
                Skills to focus on: {', '.join(extracted_skills)}
                
                Strengths: {', '.join(analysis_result.get('strengths', []))}
                
                Areas for improvement: {', '.join(analysis_result.get('missing_skills', []))}
                
                {weaknesses_text}
                
                Target role: {target_role if target_role else "Not specified"}
                """
                
    prompt = f"""
                Provide detailed suggestions to improve this resume in the following areas: {', '.join(remaining_areas)}.
                
                {context}
                
                For each improvement area, provide:
                1. A general description of what needs improvement
                2. 3-5 specific actionable suggestions
                3. Where relevant, provide a before/after example
                
                Format the response as a JSON object with improvement areas as keys, each containing:
                - "description": general description
                - "specific": list of specific suggestions
                - "before_after": (where applicable) a dict with "before" and "after" examples
                
                Only include the requested improvement areas that aren't already covered.
                Focus particularly on addressing the resume weaknesses identified.
                """
             



    response = llm.invoke(prompt)
    # try to parse json response
    ai_improvements = {}
    json_match = re.search(r'```(?:json)?\s*([\s\S]+?)\s*```', response.content)
    if json_match:
        try:
            ai_improvements = json.loads(json_match.group(1))
            # merge with existing imoprovemtns
            improvements.update(ai_improvements)
        except json.JSONDecodeError:
            pass


    


                
          
            


except Exception as e: 
    pass

    


In [116]:
ans = response.content
json_match = re.search(r'```(?:json)?\s*([\s\S]+?)\s*```', response.content)
ai_immmpo = {}
impo = {}
ai_immmpo = json.loads(json_match.group(1))
impo.update(ai_immmpo)



In [129]:
for area in improvement_areas:
    if area not in improvements:
        improvements[area] = {
                        "description": f"Improvements needed in {area}",
                        "specific": ["Review and enhance this section"]
                    }
print(improvements['Skills Highlighting'])

    

{'description': 'Your resume needs to better highlight key skills that are important for the role.', 'specific': ['**Mlops**: Quantify the impact of MLOps tools used in the projects. For example, improved model deployment speed, reduced training time, or enhanced model monitoring capabilities.', '**Mlops**: Describe the specific MLOps workflows implemented (e.g., CI/CD pipeline for model deployment, automated model retraining process, version control for models and data).', '**Mlops**: Focus on the end-to-end MLOps lifecycle, including data validation, model training, deployment, monitoring, and retraining.', '**Devops**: Quantify the impact of using DevOps tools. For example, specify reduced deployment time, improved efficiency, or enhanced scalability.', '**Devops**: Describe the infrastructure and automation aspects of the projects. Mention containerization, orchestration, CI/CD pipelines, and infrastructure-as-code.', '**Devops**: Elaborate on the role played in implementing DevOps

In [138]:
for area, content in improvements.items():
    print(f"\n📘 Area: {area}")
    print(f"📝 Description: {content['description']}")
    for item in content.get("specific", []):
        print(f"  🔹 {item}")
    if "before_after" in content:
        print("\n🧾 Before:\n", content["before_after"].get("before", ""))
        print("\n✅ After:\n", content["before_after"].get("after", ""))


📘 Area: Skills Highlighting
📝 Description: Your resume needs to better highlight key skills that are important for the role.
  🔹 **Mlops**: Quantify the impact of MLOps tools used in the projects. For example, improved model deployment speed, reduced training time, or enhanced model monitoring capabilities.
  🔹 **Mlops**: Describe the specific MLOps workflows implemented (e.g., CI/CD pipeline for model deployment, automated model retraining process, version control for models and data).
  🔹 **Mlops**: Focus on the end-to-end MLOps lifecycle, including data validation, model training, deployment, monitoring, and retraining.
  🔹 **Devops**: Quantify the impact of using DevOps tools. For example, specify reduced deployment time, improved efficiency, or enhanced scalability.
  🔹 **Devops**: Describe the infrastructure and automation aspects of the projects. Mention containerization, orchestration, CI/CD pipelines, and infrastructure-as-code.
  🔹 **Devops**: Elaborate on the role played in

In [ ]:
target_role="Data Scientist"
highlight_skills="Python, Machine Learning, SQL"
skills_to_highlight = []

if len(highlight_skills)>100:
    jd_text  = highlight_skills
    try:
        parsed_skills = extract_skills_jd(highlight_skills)
        if parsed_skills:
            skills_to_highlight = parsed_skills
        else:
            skills_to_highlight = [s.strip() for s in highlight_skills.split(",") if s.strip()]
    except:
        skills_to_highlight = [s.strip() for s in highlight_skills.split(",") if s.strip()]
else:
    skills_to_highlight = [s.strip() for s in highlight_skills.split(",") if s.strip()]







Job Title: Data Scientist
Location: Remote / Bangalore, India
Job Type: Full-time

About the Role:
We are seeking a Data Scientist who is passionate about using data to solve real-world problems and drive strategic decisions. You'll work closely with business, product, and engineering teams to uncover insights, build predictive models, and design experiments that impact millions of users.

Key Responsibilities:
- Analyze structured and unstructured data from multiple sources to extract meaningful insights.
- Build machine learning models for prediction, classification, segmentation, and recommendation.
- Design A/B tests and evaluate the performance of models and features.
- Communicate findings clearly to stakeholders using dashboards, reports, and visualizations.
- Collaborate with data engineers and software developers to deploy scalable data solutions.
- Monitor model performance and perform periodic model retraining.

Required Skills & Qualifications:
- Bachelor's or Master's deg

In [157]:
def get_improved_resume(target_role, highlight_skills, jd_text):
    """Generate an improved version of the resume optimized for the job description"""
    if not resume_text:
        return "Please upload and analyze a resume first."

    try:
        # Parse highlight skills if provided
        skills_to_highlight = []
        if highlight_skills:
            if len(highlight_skills) > 100:
                jd_text = highlight_skills
                try:
                    parsed_skills = extract_skills_jd(highlight_skills)
                    if parsed_skills:
                        skills_to_highlight = parsed_skills
                    else:
                        skills_to_highlight = [s.strip() for s in highlight_skills.split(",") if s.strip()]
                except:
                        skills_to_highlight = [s.strip() for s in highlight_skills.split(",") if s.strip()]
        else:
            skills_to_highlight = [s.strip() for s in highlight_skills.split(",") if s.strip()]

        if not skills_to_highlight and analysis_result:
            skills_to_highlight = analysis_result.get('missing_skills', [])
            skills_to_highlight.extend([
                skill for skill in analysis_result.get('strengths', [])
                if skill not in skills_to_highlight
                ])
            if extracted_skills:
                    skills_to_highlight.extend([
                        skill for skill in extracted_skills
                        if skill not in skills_to_highlight
                    ])

        weakness_context = ""
        improvement_examples = ""
        if resume_weakness:
            weakness_context = "Address these specific weaknesses:\n"
            for weakness in resume_weakness:
                skill_name = weakness.get('skill', '')
                weakness_context += f"- {skill_name}: {weakness.get('detail', '')}\n"
                if 'suggestions' in weakness and weakness['suggestions']:
                    weakness_context += "  Suggested improvements:\n"
                    for suggestion in weakness['suggestions']:
                        weakness_context += f"  * {suggestion}\n"
                if 'example' in weakness and weakness['example']:
                    improvement_examples += f"For {skill_name}: {weakness['example']}\n\n"

        llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEYY)

        jd_context = ""
        if jd_text:
            jd_context = f"Job Description:\n{jd_text}\n\n"
        elif target_role:
            jd_context = f"Target Role: {target_role}\n\n"

        prompt = f"""
            Rewrite and improve this resume to make it highly optimized for the target job.

            {jd_context}
            Original Resume:
            {resume_text}

            Skills to highlight (in order of priority): {', '.join(skills_to_highlight)}

            {weakness_context}

            Here are specific examples of content to add:
            {improvement_examples}

            Please improve the resume by:
            1. Adding strong, quantifiable achievements
            2. Highlighting the specified skills strategically for ATS scanning
            3. Addressing all the weakness areas identified with the specific suggestions provided
            4. Incorporating the example improvements provided above
            5. Structuring information in a clear, professional format
            6. Using industry-standard terminology
            7. Ensuring all relevant experience is properly emphasized
            8. Adding measurable outcomes and achievements

            Return only the improved resume text without any additional explanations.
            Format the resume in a modern, clean style with clear section headings.
            """

        response = llm.invoke(prompt)
        improved_resume = response.content.strip()

        # with tempfile.NamedTemporaryFile(delete=False, suffix='.txt', mode='w', encoding='utf-8') as tmp:
        #     tmp.write(improved_resume)
        #     improved_resume_path = tmp.name

        return improved_resume
    except Exception as e:
            print(f"Error generating improved resume: {e}")
            return "Error generating improved resume. Please try again."
    

In [158]:
target_role="Data Scientist"
highlight_skills="Python, Machine Learning, SQL"

print(get_improved_resume(target_role, highlight_skills, jd_text))

**Utkarsh Raj**
Email: utkarsh@example.com | Phone: +91-9876543210 | Location: Bangalore, India (Remote)
LinkedIn: www.linkedin.com/in/utkarsh9911 | GitHub: github.com/utkarshraj99

**Summary**

Data Scientist with 2+ years of experience leveraging machine learning, statistical analysis, and cloud technologies to drive data-informed decisions. Proven ability to build and deploy predictive models, design and analyze A/B tests, and communicate complex findings to stakeholders. Proficient in Python, SQL, and leading machine learning frameworks. Passionate about applying MLOps and DevOps principles to improve model performance and deployment efficiency.

**Skills**

*   **Programming Languages:** Python, SQL
*   **Machine Learning:** Scikit-learn, TensorFlow, PyTorch, Regression, Classification, Clustering, Time Series Analysis, NLP
*   **Data Analysis & Visualization:** Pandas, NumPy, Matplotlib, Seaborn, Tableau
*   **Cloud Computing:** Google Cloud Platform (GCP), Google Cloud Storage
*

In [43]:
class person:
    def __init__(self, name: str, course:str, address:int):
        self.name = name

        self.__course = course
        self._address = address
        # self.__address = address

    def getinfo(self):
        return self.name, self._address, self.__course

ab = person('rakesh', 'btech', 1234)
ans = ab
print(ans.name)
# for i in ans:
#     print(i)
print(ab._address)

    

        

rakesh
1234


In [52]:
class dog:
    def __init__(self, breed, color):
        self.breed = breed
        self.color = color
    def sound(self):
        return 'woof'

class anotherbreed(dog):
    def __init__(self, name,breed,color):
        super().__init__(breed,color)
        self.name = name
        self.sound()
    def getinfo(self):
        return self.name, self.breed, self.color, self.sound()

ab = anotherbreed('pitbull', 'wild', 'brown')    
for ans in ab.getinfo():
    print(ans)
    
    

pitbull
wild
brown
woof
